In [11]:
#import's
from dotenv import load_dotenv
import os
import csv
import requests
import time
from datetime import datetime, timedelta 
#Loading API key's from .env file
load_dotenv()
api_key = os.getenv("API_KEY")
base_url = os.getenv("BASE_URL")
IATA_CODE = "MCI" #Kansas City
TYPE = "arrival"
#this code is so the API automatically pull's next's weeks flight schedule
start_date = datetime.today() + timedelta(days=8)
date_range = [start_date + timedelta(days=i) for i in range(7)]

In [12]:
all_flights_week = []
# this is date loop for the API so each day of the week get's called
for date in date_range:
    formatted_date = date.strftime("%Y-%m-%d")
    params = {
        "iataCode": IATA_CODE,
        "type": TYPE,
        "date": formatted_date,
        "access_key": api_key
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json().get("data", [])
        if data:
            for flight in data:
                flight["flight_date"] = formatted_date 

            all_flights_week.extend(data)
            print(f"✅ Retrieved {len(data)} flights")
        else:
            print(f"⚠️ No data returned for {formatted_date}")
    else:
        print(f"❌ Error {response.status_code}: {response.text}")
        
        break
# Time is a full minute between each loop because that has to with the rate limit, 
# the loop is also designed to break if it fails and then the data is supposed to be dropped
    
    time.sleep(60)







✅ Retrieved 251 flights
✅ Retrieved 280 flights
✅ Retrieved 201 flights
✅ Retrieved 252 flights
✅ Retrieved 242 flights
✅ Retrieved 280 flights
✅ Retrieved 248 flights


In [13]:
import pandas as pd
from pandas import json_normalize

In [14]:
#it was a pretty clean JSON file so not diffcult to parse with a standard JSON_normalize

df = json_normalize(all_flights_week, sep='_')
columns_to_keep = [
    'flight_date',
    'weekday',
    'arrival_iataCode',
    'arrival_terminal',
    'arrival_scheduledTime',
    'departure_iataCode',
    'airline_name',
    'flight_iataNumber',
    'aircraft_modelText'
]

df_filtered = df[columns_to_keep]


df.head()

,weekday,flight_date,departure_iataCode,departure_icaoCode,departure_terminal,departure_gate,departure_scheduledTime,arrival_iataCode,arrival_icaoCode,arrival_terminal,...,airline_icaoCode,flight_number,flight_iataNumber,flight_icaoNumber,codeshared_airline_name,codeshared_airline_iataCode,codeshared_airline_icaoCode,codeshared_flight_number,codeshared_flight_iataNumber,codeshared_flight_icaoNumber
0,3,2025-07-30,phx,kphx,4,,18:55,mci,kmci,,...,jag,4285,as4285,jag4285,american airlines,aa,aal,623,aa623,aal623
1,3,2025-07-30,phx,kphx,4,,18:55,mci,kmci,,...,aal,623,aa623,aal623,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2025-07-30,atl,katl,s,,08:20,mci,kmci,,...,lan,6120,la6120,lan6120,delta air lines,dl,dal,2471,dl2471,dal2471
3,3,2025-07-30,atl,katl,s,,08:20,mci,kmci,,...,wja,7243,ws7243,wja7243,delta air lines,dl,dal,2471,dl2471,dal2471
4,3,2025-07-30,atl,katl,s,,08:20,mci,kmci,,...,dal,2471,dl2471,dal2471,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# then a time stamp is created to perserve a file with todays data, so it can be reference in historic sense
timestamp = datetime.now().strftime('%Y%m%d_%H%M')
raw_path = f"flight_data/raw/flight_arrivals_{timestamp}.csv"
df_filtered.to_csv(raw_path, index=False)

